<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Parameters Import Test </span> </p>

Uncomment the next cells to run or debug this test.

In [0]:
//#!eval-notebook "../Initialization/InitSystemorphRefDataToMemory"

In [0]:
//#!eval-notebook "../Initialization/InitSystemorphUtils"

In [0]:
//var pathToFiles = "../Files/";
//await Import.InitializeDataNodesAsync(DataSource, pathToFiles, new string[] {"CH"});

In [0]:
DataSource.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>());

# Setting Constants

In [0]:
var EnableScenarioBackup = Scenarios.EnableScenario;

In [0]:
Scenarios.EnableScenario = true;

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenarioParameter = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "DUP10pct", ImportFormats.Cashflow);

In [0]:
var estimateTypes = (await DataSource.Query<EstimateType>().ToArrayAsync());
var actualEstimateTypes = estimateTypes.Where(et => et.InputSource.Contains(InputSource.Actual)).Select(x => x.SystemName);
var cashflowEstimateTypes = estimateTypes.Where(et => et.InputSource.Contains(InputSource.Cashflow)).Select(x => x.SystemName);
var actualAndCashflowEstimateTypes = actualEstimateTypes.Intersect(cashflowEstimateTypes);
var onlyActualEstimateTypes = actualEstimateTypes.Except(cashflowEstimateTypes);
var onlyCashflowEstimateTypes = cashflowEstimateTypes.Except(actualEstimateTypes);

# Best Estimate initialization

In [0]:
var cashflowBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,CU,P,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,CU,P,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
await Import.FromString(cashflowBestEstimateInit).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
var actualsBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.1,CF,NIC,A,,-280
DT1.1,CF,PR,A,,400
DTR1.1,CF,PR,A,,-200
DTR1.1,CF,NIC,A,,140";

In [0]:
await Import.FromString(actualsBestEstimateInit).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateInit = await DataSource.Query<IfrsVariable>().ToArrayAsync();

# Scenario parameters import

In [0]:
var parametersScenarios = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,DUP10pct
@@SingleDataNodeParameter
DataNode,PremiumAllocation
DT1.1,0.52
@@InterDataNodeParameter
DataNode,LinkedDataNode,ReinsuranceCoverage
DT1.1,DTR1.1,0.62";

In [0]:
await Import.FromString(parametersScenarios).WithFormat(ImportFormats.DataNodeParameter).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioParameter);
var ivsScenarioParameter = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioParameter.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance()).ToArray();

In [0]:
diffs.Count().Should().NotBe(0);
diffs.Where(x => x.EstimateType == EstimateTypes.C).Count().Should().NotBe(0);

# Re-import of the same parameters should not trigger calculations 

In [0]:
var log = await Import.FromString(parametersScenarios).WithFormat(ImportFormats.DataNodeParameter).WithTarget(DataSource).ExecuteAsync();
log

In [0]:
((log.Warnings.FirstOrDefault()?.ToString().Remove(0,40).Remove(125,2)?? "") == Warning.VariablesAlreadyImported.GetMessage()).Should().Be(true);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioParameter);
var ivsScenarioParameterSameReimport = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioParameterSameReimport.Except(ivsScenarioParameter, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().Be(0);

# Re-import of best estimate cashflow

In [0]:
var cashflowBestEstimateReImport = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,131,0,0,111,0,0,111,0,0,111,0,0,0,111,0,0,111,0,0,111,0,0,111,0
DT1.1,NIC,BE,CL,C,,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31,-31
DT1.1,CU,P,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6,-2.6
DTR1.1,PR,BE,CL,C,,51,0,0,51,0,0,51,0,0,51,0,0,51,0,0,51,0,0,51,0,0,51,0,0
DTR1.1,NIC,BE,CL,C,,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
DTR1.1,CU,P,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31";

In [0]:
await Import.FromString(cashflowBestEstimateReImport).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

## Best Estimate change

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateReimport = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsBestEstimateReimport.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

## Scenario should change

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioParameter);
var ivsScenarioParameterReimportBestEstimateCashflow = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioParameterReimportBestEstimateCashflow.Except(ivsScenarioParameter, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

# Re-import of scenario parameters

In [0]:
var parametersScenariosReimport = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,DUP10pct
@@SingleDataNodeParameter
DataNode,PremiumAllocation
DT1.1,0.23
@@InterDataNodeParameter
DataNode,LinkedDataNode,ReinsuranceCoverage
DT1.1,DTR1.1,0.23";

In [0]:
await Import.FromString(parametersScenariosReimport).WithFormat(ImportFormats.DataNodeParameter).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioParameter);
var ivsScenarioParameterReimport = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioParameterReimport.Except(ivsScenarioParameterReimportBestEstimateCashflow, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

# Restore constants

In [0]:
Scenarios.EnableScenario = EnableScenarioBackup;